<center><img src="Logo PI MLOps STEAM.png" alt="Logo PI MLOps STEAM"></center>

#### Importo los df para trabajar

In [1]:
# Importo librerías.
import pandas as pd

# Rutas a los DataFrames procesados en CSV
ruta_steam_games_csv = r'C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI MLOps - STEAM 01\Df procesados en csv\steam_games.csv'
ruta_user_reviews_csv = r'C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI MLOps - STEAM 01\Df procesados en csv\user_reviews.csv'
ruta_users_items_csv = r'C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI MLOps - STEAM 01\Df procesados en csv\users_items.csv'

# Convierte los archivos CSV en DataFrames de pandas
df_steam_games = pd.read_csv(ruta_steam_games_csv)
df_user_reviews = pd.read_csv(ruta_user_reviews_csv)
df_users_items = pd.read_csv(ruta_users_items_csv)


Visualizo las columnas

In [2]:
print(df_steam_games.columns)
print(df_user_reviews.columns)
print(df_users_items.columns)
Index(['publisher', 'app_name', 'title', 'año de lanzamiento', 'año', 'price',
       'user_id', 'developer', 'genres_Action_new', 'genres_Adventure_new',
       'genres_Indie_new', 'tags_Action_new', 'tags_Adventure_new',
       'tags_Casual_new', 'tags_Indie_new'],
      dtype='object')
Index(['user_id', 'item_id', 'recommend', 'coincide'], dtype='object')
Index(['user_id', 'items_count', 'steam_id', 'user_url', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')


Index(['publisher', 'app_name', 'title', 'año de lanzamiento', 'año', 'price',
       'user_id', 'developer', 'genres_Action_new', 'genres_Adventure_new',
       'genres_Indie_new', 'tags_Action_new', 'tags_Adventure_new',
       'tags_Casual_new', 'tags_Indie_new'],
      dtype='object')
Index(['user_id', 'item_id', 'recommend', 'coincide'], dtype='object')
Index(['user_id', 'items_count', 'steam_id', 'user_url', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')


#### Creo 'Sentiment_analysis'

In [10]:

def analyze_sentiment(item_id):
    """
    Analizo el sentimiento de un ítem y devuelve un valor en la escala 0 (malo), 1 (neutral) o 2 (positivo).
    """
    # Convierto a cadena si 'item_id' no es una cadena
    item_id_str = str(item_id) if not isinstance(item_id, str) else item_id
    
    # Simulo el análisis de sentimiento e implemento aquí tu lógica real con NLP para clasificar las reseñas.
    return 0 if "malo" in item_id_str.lower() else 1 if "neutral" in item_id_str.lower() else 2

if 'item_id' in df_user_reviews.columns:
    df_user_reviews['sentiment_analysis'] = df_user_reviews['item_id'].apply(analyze_sentiment)
else:
    print("La columna 'item_id' no está presente en df_user_reviews.")


#### Verifico el tipo de dato en cada columna.

In [21]:

def verificar_tipo_datos(df):
    """
    Verifico el tipo de datos en cada columna de un DataFrame y devuelve un resumen.
    """
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}
    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
    return df_info

# Visualizo
print(verificar_tipo_datos(df_steam_games))


            nombre_campo                        tipo_datos  no_nulos_%  \
0              publisher  [<class 'str'>, <class 'float'>]       99.92   
1               app_name                   [<class 'str'>]      100.00   
2                  title                   [<class 'str'>]      100.00   
3     año de lanzamiento                   [<class 'str'>]      100.00   
4                    año                   [<class 'int'>]      100.00   
5                  price                   [<class 'str'>]      100.00   
6                user_id                   [<class 'int'>]      100.00   
7              developer                   [<class 'str'>]      100.00   
8      genres_Action_new                   [<class 'int'>]      100.00   
9   genres_Adventure_new                   [<class 'int'>]      100.00   
10      genres_Indie_new                   [<class 'int'>]      100.00   
11       tags_Action_new                   [<class 'int'>]      100.00   
12    tags_Adventure_new              

#### Verifico duplicados por columna.

In [22]:

def verifica_duplicados_por_columna(df, columna):
    """
    Verifico duplicados en un DataFrame por una columna específica y devuelve los resultados ordenados.
    """
    duplicated_rows = df[df.duplicated(subset=columna, keep=False)]
    if duplicated_rows.empty:
        return "No hay duplicados"
    
    duplicated_rows_sorted = duplicated_rows.sort_values(by=columna)
    return duplicated_rows_sorted


#### Configuro FastAPI para simular los endpoints.

In [ ]:

# Configuración de la API FastAPI
app = FastAPI()


Endpoint obtención año de lanzamiento con más horas jugadas para cada género.

In [41]:
@app.get("/playtime_genre/{genero}")
async def playtime_genre(genero: str):
    genre_data = df_steam_games[df_steam_games[f"genres_{genero}_new"] == 1]

    if genre_data.empty:
        raise HTTPException(status_code=404, detail=f"No hay datos para el género {genero}")

    genre_data['user_id'] = genre_data['user_id'].astype('object')

    merged_data = pd.merge(genre_data, df_users_items, on='user_id', how='inner')
    year_with_most_playtime = merged_data.groupby('año de lanzamiento')['playtime_forever'].sum().idxmax()

    return {"Año de lanzamiento con más horas jugadas para " + genero: int(year_with_most_playtime)}


Endpoint para obtener el usuario con más horas jugadas y las horas jugadas por año de lanzamiento para un género

In [42]:
@app.get("/user_for_genre/{genero}", response_model=dict)
def UserForGenre(genero: str):
    """
    Devuelve el usuario con más horas jugadas y las horas jugadas por año de lanzamiento para un género específico.
    """
    # Lógica para el endpoint
    # Utiliza información de df_steam_games y df_users_items
    # (Ejemplo de retorno)
    example_output = {
        "Usuario con más horas jugadas para " + genero: "us213ndjss09sdf",
        "Horas jugadas": [
            {"Año": 2013, "Horas": 203},
            {"Año": 2012, "Horas": 100},
            {"Año": 2011, "Horas": 23}
        ]
    }
    return JSONResponse(content=example_output)


Endpoint para obtener el top 3 de juegos MÁS recomendados por usuarios para un año dado

In [43]:
@app.get("/users_recommend/{año}", response_model=list)
def UsersRecommend(año: int):
    """
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para un año dado.
    """
    # Lógica para el endpoint
    # Utiliza información de df_user_reviews
    # (Ejemplo de retorno)
    example_output = [
        {"Puesto 1": "X"},
        {"Puesto 2": "Y"},
        {"Puesto 3": "Z"}
    ]
    return JSONResponse(content=example_output)


Endpoint para obtener el top 3 de juegos MENOS recomendados por usuarios para un año dado

In [44]:
@app.get("/users_not_recommend/{año}", response_model=list)
def UsersNotRecommend(año: int):
    """
    Devuelve el top 3 de juegos MENOS recomendados por usuarios para un año dado.
    """
    # Lógica para el endpoint
    # Utiliza información de df_user_reviews
    # (Ejemplo de retorno)
    example_output = [
        {"Puesto 1": "X"},
        {"Puesto 2": "Y"},
        {"Puesto 3": "Z"}
    ]
    return JSONResponse(content=example_output)

Endpoint para obtener el análisis de sentimiento para un año dado.


In [45]:
@app.get("/sentiment_analysis/{año}", response_model=dict)
def sentiment_analysis(año: int):
    """
    Devuelve el análisis de sentimiento para un año dado.
    """
    # Lógica para el endpoint
    # Utiliza información de df_user_reviews
    # (Ejemplo de retorno)
    example_output = {'Negativo': 182, 'Neutro': 120, 'Positivo': 278}
    return JSONResponse(content=example_output)
